In [1]:
import sys
assert sys.version_info >= (3, 5)

In [2]:
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

import numpy as np

In [2]:
import os

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="1"

#check os.environ ld_library_path is the same here as when I do it in python via terminal, if I get issues

#sometimes I can't select the GPU. In this case, try: https://forums.fast.ai/t/tip-limiting-tensorflow-to-one-gpu/1995

In [5]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [6]:
tf.__version__


'2.1.0'

In [3]:
K = keras.backend


In [4]:
mnist = keras.datasets.mnist
(X_train_full, y_train_full), (X_test,y_test) = mnist.load_data()

X_valid, X_train = X_train_full[:5000] / 255.0, X_train_full[5000:] / 255.0

y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

X_test = X_test / 255.0

#1000 labelled points in train set for this example

X_train_la = X_train[:1000]
X_train_un = X_train[1000:]

y_train_la = y_train[:1000]
y_train_un = np.empty((X_train_la.shape[0]))



# Encoder #

In [ ]:
tf.random.set_seed(42)
np.random.seed(42)

class Sampling(keras.layers.Layer):
    def call(self, inputs):
        mean, log_var = inputs
        return K.random_normal(tf.shape(log_var)) * K.exp(log_var/2) + mean

codings_size = 50

inputs = keras.layers.Input(shape=[28, 28])
z = keras.layers.Flatten()(inputs)
z = keras.layers.Dense(600, activation="softplus")(z)
z = keras.layers.Dense(300, activation="softplus")(z)
codings_mean = keras.layers.Dense(codings_size)(z)
codings_log_var = keras.layers.Dense(codings_size)(z)
codings = Sampling()([codings_mean, codings_log_var])
variational_encoder = keras.models.Model(
    inputs=[inputs], outputs=[codings_mean, codings_log_var, codings])

# Decoder #

In [1]:
decoder_inputs = keras.layers.Input(shape=[codings_size])
x = keras.layers.Dense(300, activation="softplus")(decoder_inputs)
x = keras.layers.Dense(600, activation="softplus")(x)
x = keras.layers.Dense(28 * 28, activation="sigmoid")(x)
outputs = keras.layers.Reshape([28, 28])(x)
variational_decoder = keras.models.Model(inputs=[decoder_inputs], outputs=[outputs])

NameError: name 'keras' is not defined

# VAE # 

In [ ]:
_, _, codings = variational_encoder(inputs)
reconstructions = variational_decoder(codings)
variational_ae = keras.models.Model(inputs=[inputs], outputs=[reconstructions])

latent_loss = -0.5 * K.sum(
    1 + codings_log_var - K.exp(codings_log_var) - K.square(codings_mean),
    axis=-1)
variational_ae.add_loss(K.mean(latent_loss) / 784)
#question - how do I know which axis is being summed over? question on how loss is computed too..
variational_ae.compile(loss="binary_crossentropy", optimizer="rmsprop")

In [20]:


early_stopping_cb = keras.callbacks.EarlyStopping(patience=10,restore_best_weights=True)

history = variational_ae.fit(X_train, X_train, epochs=200, batch_size=32,
                             validation_data=(X_valid, X_valid),  callbacks=[early_stopping_cb])

Train on 55000 samples, validate on 5000 samples
Epoch 1/200
55000/55000 [==============================] - 7s 132us/sample - loss: 0.2110 - val_loss: 0.1755
Epoch 2/200
55000/55000 [==============================] - 6s 116us/sample - loss: 0.1671 - val_loss: 0.1573
Epoch 3/200
55000/55000 [==============================] - 6s 115us/sample - loss: 0.1546 - val_loss: 0.1497
Epoch 4/200
55000/55000 [==============================] - 6s 115us/sample - loss: 0.1493 - val_loss: 0.1467
Epoch 5/200
55000/55000 [==============================] - 6s 115us/sample - loss: 0.1468 - val_loss: 0.1446
Epoch 6/200
55000/55000 [==============================] - 6s 116us/sample - loss: 0.1451 - val_loss: 0.1470
Epoch 7/200
55000/55000 [==============================] - 6s 118us/sample - loss: 0.1437 - val_loss: 0.1435
Epoch 8/200
55000/55000 [==============================] - 6s 117us/sample - loss: 0.1425 - val_loss: 0.1418
Epoch 9/200
55000/55000 [==============================] - 6s 114us/sample - lo

# SVM Classifier #

VAE is trained on entire set, and then SVM is trained on the coding of the labelled data.

In [21]:
_,_,codings = variational_encoder.predict(X_train_la)

from sklearn.svm import SVC

svc_class = SVC(kernel="rbf", gamma='scale', random_state=42, C=1000)
svc_class.fit(codings, y_train_la)
svc_class.score(codings, y_train_la)


1.0

In [22]:
_,_,codings2 = variational_encoder.predict(X_test)



In [23]:
svc_class.score(codings2, y_test)


0.8272